In [54]:
import sys
sys.path.append("..") # append the directory above where the databese is.
import pandas as pd
import numpy as np
import sqlite3
import time
import numba
%load_ext line_profiler
### disable scientific notation in pandas
pd.set_option('display.float_format', '{:.2f}'.format) ### display up to 2 decimal pts

In [ ]:
connection = sqlite3.connect("../db.sqlite3")
cursor = connection.cursor() 
cursor.execute("SELECT * FROM CountryIndicators")  
country_list = cursor.fetchall() # we get the 
np.shape(country_list)
print(country_list[:3]) # okay it has the same format as the table where the rows are tuples in this list

In [ ]:
df = pd.DataFrame(country_list, columns=["CountryCode", "IndicatorCode", "Year", "Value" ])

In [ ]:
df_ARB = df[df["CountryCode"] == "ARB"]
print(df_ARB["IndicatorCode"].nunique())
df_one_ind = df_ARB[df["IndicatorCode"] == "SP.ADO.TFRT"][["Year", "Value"]]
df_one_ind = df_one_ind.rename(columns={"Value":"indicator"})

df_another_ind = df_ARB[df["IndicatorCode"] == "SP.POP.DPND"][["Year", "Value"]]
df_another_ind = df_another_ind.rename(columns={"Value":"another_indicator"})
df_merged = df_one_ind.merge(df_another_ind, left_on="Year", right_on="Year")
 #okay this works so
print(df_merged)


In [81]:

40 == df_ARB[df_ARB["IndicatorCode"] == "AG.AGR.TRAC.NO"].groupby("IndicatorCode").count()["Year"]

#for indicator in df_ARB["IndicatorCode"]:
    #print(indicator)

IndicatorCode
AG.AGR.TRAC.NO    True
Name: Year, dtype: bool

In [131]:
def uniques(df, indicator):
    """ returns true or false if the indicator is in every year"""
    indicator_range = np.array(df[df["IndicatorCode"] == indicator]["Year"])
    year_range = np.array(range(df["Year"].min(), df["Year"].max() + 1))
    if len(year_range) != len(indicator_range):
        return False
    return True
def create_total_df(df):
    n = 0
    df_final = pd.DataFrame(df["Year"])
   # print(df_final)
    #print(df["IndicatorCode"].nunique())
    for indicator in df["IndicatorCode"].unique():
        #print(f"{indicator}")
        if uniques(df, indicator):
            df_tmp = df[df["IndicatorCode"] == f"{indicator}"][["Year", "Value"]]
            df_tmp = df_tmp.rename(columns={"Value":f"{indicator}"})
            df_final = df_final.merge(df_tmp, left_on="Year", right_on="Year")
    return df_final

#df_s = []
%lprun -f create_total_df create_total_df(df_ARB[df_ARB["Year"] > 1999])
#ncountrys = df["CountryCode"].nunique()
#count = 0
#for country in df["CountryCode"].unique():
 #   s = time.time()
  #  df_s.append(create_total_df(df[df["CountryCode"] == country]))
   # count += 1
    #print(country)
    #print(f"ETA: {(time.time() - s) *(ncountrys - 1)/60} min")


Timer unit: 1e-06 s

Total time: 39.7733 s
File: <ipython-input-131-7f26fdef6387>
Function: create_total_df at line 8

Line #      Hits         Time  Per Hit   % Time  Line Contents
     8                                           def create_total_df(df):
     9         1          3.0      3.0      0.0      n = 0
    10         1       2973.0   2973.0      0.0      df_final = pd.DataFrame(df["Year"])
    11                                              # print(df_final)
    12                                               #print(df["IndicatorCode"].nunique())
    13       625       3636.0      5.8      0.0      for indicator in df["IndicatorCode"].unique():
    14                                                   #print(f"{indicator}")
    15       624    1410615.0   2260.6      3.5          if uniques(df, indicator):
    16       374    1191715.0   3186.4      3.0              df_tmp = df[df["IndicatorCode"] == f"{indicator}"][["Year", "Value"]]
    17       374     399953.0   1069.4  